In [29]:
import psycopg2 as psy

from config import config

In [30]:
def connectTest():
    

    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psy.connect(**config()) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psy.DatabaseError, Exception) as error:
        print(error)

connectTest()

Connected to the PostgreSQL server.


<connection object at 0x7eff85d0efc0; dsn: 'user=postgres password=xxx dbname=dummy host=localhost', closed: 0>

In [31]:
def connect():
    conn = None
    try:
        conn = psy.connect(**config())
        print("Connected to the PostgreSQL server")

        crsr = conn.cursor()
        crsr.execute('SELECT version()')
        db_version = crsr.fetchone()
        crsr.close()
        print(f"PostgreSQL database version: {db_version}")

    except(Exception, psy.DatabaseError) as error:
        print(error)

    finally:
        if conn != None:
            conn.close()
            print("Connection closed")

connect()


Connected to the PostgreSQL server
PostgreSQL database version: ('PostgreSQL 16.6 (Ubuntu 16.6-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.2.0-23ubuntu4) 13.2.0, 64-bit',)
Connection closed
